# **--- `WfMi` : Measuring information ---**
---

In this tutorial, we're going to go through the following points :
1. **[$I(continuous, discret)$]** = Measure the quantity of information shared between brain data and stimulus (or outcome) types (\~decoding)
2. **[$I(continuous, continuous)$]** = Measure the quantity of information shared between brain data and a continuous variable (e.g. behavioral model like PE, the reaction time etc.) (\~regression)
4. How to handle the spatial dimension
    * Results at the single contact / channel level?
    * Results inside a brain region by grouping the data coming from multiple channels?
5. How to define a custom estimator of information

In [ ]:
import os

import numpy as np
import xarray as xr
import pandas as pd

from mne.utils import ProgressBar

from frites.dataset import DatasetEphy
from frites.workflow import WfMi

import matplotlib.pyplot as plt

---
# **0 - Functions**

In [ ]:
###############################################################################
###############################################################################
#                 Load the data of a single subject
###############################################################################
###############################################################################

def load_ss(root, subject_nb):
    """Load the data of a single subject.
    
    Parameters
    ----------
    root : string
        Root path where the dataset is stored
    subject_nb : int
        Subject number [0, 12]
    
    Returns
    -------
    hga : xarray.DataArray
        Xarray containing the high-gamma activity
    anat : pandas.DataFrame
        Table containing the anatomical informations
    beh : pandas.DataFrame
        Table containing the behavioral informations
    """
    # load the high-gamma activity
    file_hga = os.path.join(root, 'hga', f'hga_s-{subject_nb}.nc')
    hga = xr.load_dataarray(file_hga)

    # load the name of the brain regions
    file_anat = os.path.join(root, 'anat', f'anat_s-{subject_nb}.xlsx')
    anat = pd.read_excel(file_anat)

    # load the behavior
    file_beh = os.path.join(root, 'beh', f'beh_s-{subject_nb}.xlsx')
    beh = pd.read_excel(file_beh)
    
    return hga, anat, beh


###############################################################################
###############################################################################
#                 Load the data of multiple subjects
###############################################################################
###############################################################################

def load_ms(root, s_range=[0, 11], model='outcome', condition='rew',
            space='channels', mean_roi=True, prepend_suj_to_ch=True):
    """Load multiple subjects.
    
    Parameters
    ----------
    root : string
        Root path where the dataset is stored
    s_range : int or list
        Subjects to load. Use either an integer (e.g. 7) to load a single
        subject or a range of subjects (e.g. [5, 10])
    model : {'outcome', 'pe', 'rt'}
        Model to use. Use either :
        
            * 'outcome' : find differences in the neural activity between the
              outcomes
            * 'pe' : find regions with an activity correlating with the
              prediction error
            * 'rt' : find regions with an activity correlating with the
              reaction time
    condition : {'rew', 'pun', 'context', 'null'}
        Condition to load. Use either :
        
            * 'rew' : for outcomes {+0€; +1€}
            * 'pun' : for outcomes {-1€; -0€}
            * 'context' : for outcomes {-1€; +1€}
            * 'null' : for outcomes {-0€; +0€}
    space : {'channels', 'roi'}
        Specify if the spatial dimension should be described with channel names
        or with brain region names
    mean_roi : bool
        Specify if you want to take the mean high-gamma activity inside a brain
        region
    prepend_suj_to_ch : bool
        Add subject name to each channel name
    
    Returns
    -------
    hga : list
        List of high-gamma activity across subjects
    """
    # inputs checking
    if isinstance(s_range, int):
        s_range = [s_range, s_range]
    s_range[1] += 1
    s_range[0], s_range[1] = max(s_range[0], 0), min(s_range[1], 12)
    mesg = f"Subject %i | model={model} | condition={condition} | space={space}"
    pbar = ProgressBar(range(s_range[0], s_range[1]), mesg=mesg % 0)
    model = model.lower()
    assert space in ['channels', 'parcels', 'roi']
    
    # get the code of the condition
    outc = {
        'rew': (+1, +2),
        'pun': (-2, -1),
        'context': (-2, +2),
        'null': (-1, +1)
    }[condition]
    
    # get the behavioral column to use
    col = {
        'outcome': 'code',
        'pe': 'PE',
        'rt': 'RT'
    }[model]
    
    # load the data
    hga = []
    for n_s in range(s_range[0], s_range[1]):
        pbar._tqdm.desc = mesg % n_s
        # load the data of a single subject
        _hga, _anat, _beh = load_ss(root, n_s)
        _outc = _hga['trials'].data
        _ch = _hga['channels'].data
        
        # replace trial dimension with the model
        _hga = _hga.rename(trials=model)
        _hga[model] = list(_beh[col])
        
        # get which outcome to keep
        keep_outc = np.logical_or(_outc == outc[0], _outc == outc[1])
        _hga = _hga[keep_outc, ...]
        
        # replace with brain regions
        if space in ['parcels', 'roi']:
            _hga = _hga.rename(channels=space)
            _hga[space] = list(_anat['roi'])
            
            # take the mean of the hga per parcel
            if mean_roi:
                _hga = _hga.groupby(space).mean(space)
        elif prepend_suj_to_ch and (space == 'channels'):
            # prepend subject number to channel name
            _hga['channels'] = [f"suj{n_s}/{c}" for c in _ch]
        
        # ascontinuous array
        _hga.data = np.ascontiguousarray(_hga.data)
        
        hga.append(_hga)
        pbar.update_with_increment_value(1)

    return hga

# **1. Loading multi-subjects**
## 1.1 Loading one or multiple subjects

In [ ]:
###############################################################################
root = '/home/etienne/DATA/Study/PBLT/gdr/'
###############################################################################

# loading a single subject
# load_ms(root, s_range=0)
# load_ms(root, s_range=7)

# loading multiple subjects
# load_ms(root, s_range=[0, 1])
# load_ms(root, s_range=[3, 6])

## 1.2 Choosing the model

- $I(continuous, discret)$
    * `'outcomes'` = **what are the contacts / brain regions for which the brain activity is different according to the outcome?**
- $I(continuous, continuous)$
    * `'pe'` (_prediction error_) = **what are the contacts / brain regions that correlates with the prediction error?**
    * `'rt'` (_reaction time_) = **what are the contacts / brain regions that correlates with the reaction time?**

In [ ]:
# seek for differences of activity between outcomes
# hga = load_ms(root, s_range=[0, 3], model='outcome')

# seek for correlation with the reaction time
# hga = load_ms(root, s_range=[0, 3], model='rt')

# seek for correlation with the prediction error
# hga = load_ms(root, s_range=[0, 3], model='pe')

## 1.3 Choosing the condition

Here, we only select the data for a subset of possible conditions, namely :
- **Reward condition     =** $outcomes \in \{"+0€", "+1€"\}$
- **Punishment condition =** $outcomes \in \{"-1€", "-0€"\}$
- **Context detection    =** $outcomes \in \{"-1€", "+1€"\}$
- **Null comparison      =** $outcomes \in \{"-0€", "+0€"\}$

Therefore, you can ask questions like :
- **Model-free analysis** (contrasting conditions, \~decoding)
    - Are there differences of HGA according to the outcome during the reward of punishment conditions?
    - What are the brain regions that can differentiate the context?
    - Are there regions capable of finding differences between the 0€ of both conditions?
- **Model-based analysis** (correlation with a model, \~regression)
- What are the brain regions correlating with the reward or punishment prediction error?
- Are there brain regions for which the brain activity correlates with the reaction time during the reward or punishment conditions?

### 1.3.1 Model-free analysis

In [ ]:
# Reward = differences of hga between outcomes (+0€; +1€)
# hga = load_ms(root, s_range=[0, 3], model='outcome', condition='rew')

# Punishment = differences of hga between outcomes (-1€; -0€)
# hga = load_ms(root, s_range=[0, 3], model='outcome', condition='pun')

# Context = differences of hga between outcomes (-1€; +1€)
# hga = load_ms(root, s_range=[0, 3], model='outcome', condition='context')

# Null = differences of hga between outcomes (-0€; +0€)
# hga = load_ms(root, s_range=[0, 3], model='outcome', condition='null')

### 1.3.2 Model-based analysis

In [ ]:
# brain regions correlating with the reward prediction error
# hga = load_ms(root, s_range=[0, 3], model='pe', condition='rew')

# brain regions correlating with the punishment prediction error
# hga = load_ms(root, s_range=[0, 3], model='pe', condition='pun')

# brain regions correlating with the reaction time during the reward condition
# hga = load_ms(root, s_range=[0, 3], model='rt', condition='rew')

# brain regions correlating with the reaction time during the punishment condition
# hga = load_ms(root, s_range=[0, 3], model='rt', condition='pun')

## 1.4 Choosing the spatial scale : channels or brain regions?
### 1.4.1 Computations at the scale of the channel / contact

In [ ]:
# hga = load_ms(root, s_range=0, space='channels')

### 1.4.1 Computations at the scale of the brain region

In [ ]:
# use the brain region names to caracterize the space
# hga = load_ms(root, s_range=7, space='roi', mean_roi=False)

# same, but this time we also take the mean of the hga inside each brain region
# hga = load_ms(root, s_range=7, space='roi', mean_roi=True)

# **2. Measure information**
## 2.1 $I(continuous; discret)$ = Model-free analysis

In [ ]:
# load the hga, with the outcomes during the reward condition
hga = load_ms(root, model='outcome', condition='rew', space='roi')

# define the DatasetEphy
ds = DatasetEphy(hga, y='outcome', roi='roi', times='times')

"""
define the type of mutual information to compute
'cd' = I(continuous; discret)
     = I(brain data; stimulus)
"""
mi_type = 'cd'

# define a workflow of mutual-information
wf = WfMi(mi_type=mi_type, inference='ffx')

# run the workflow (without stat for the moment)
mi, pv = wf.fit(ds, mcp=None)

# plot the result
plt.figure(figsize=(10, 8))
mi.plot(x='times', hue='roi')
plt.axvline(0., color='k')
plt.title(r"$I(data; \{+0€; +1€\})$");

## 2.2 $I(continuous; continuous)$ = Model-based analysis

In [ ]:
# load the hga, with the outcomes during the reward condition
hga = load_ms(root, model='pe', condition='rew', space='roi')

# define the DatasetEphy
ds = DatasetEphy(hga, y='pe', roi='roi', times='times')

"""
define the type of mutual information to compute
'cc' = I(continuous; continuous)
     = I(brain data; PE)
"""
mi_type = 'cc'

# define a workflow of mutual-information
wf = WfMi(mi_type=mi_type, inference='ffx')

# run the workflow (without stat for the moment)
mi, pv = wf.fit(ds, mcp=None)

# plot the result
plt.figure(figsize=(10, 8))
mi.plot(x='times', hue='roi')
plt.axvline(0., color='k')
plt.title(r"$I(data; RPE)$");

# **3. How to handle the spatial dimension?**

<div class="alert alert-info"><p>

**THIS IS A VERY IMPORTANT SECTION !**
</p></div>

Inside a brain region, there's multiple sEEG contacts. A natural question is **how to handle all of those contacts and avoid loosing the very specific and precious information they contained?**

You've several strategies :
1. You can average the activity within a brain region (as we did above)
2. You can try to find significant activations at the contact level, for each subject
3. You can concatenate the activity across contacts inside a brain region
4. You can use more elaborated statistics to model how the information is distributed across contacts inside a brain region (Random-Effect at the contact level)

It's hard to recommand a single method, because, as always, it's data dependent. However, as a rule of thumb, I would advise avoiding meaning the activity **before estimating the information** as much as possible. It's probably better to extract the information at the single-contact level.

## 3.1 Extract the information on the mean activity inside a brain region

In [ ]:
# load data and build the DatasetEphy
hga = load_ms(root, s_range=[0, 3], model='pe', condition='rew', space='roi',
              mean_roi=True)
ds = DatasetEphy(hga, y='pe', roi='roi', times='times')

# measure information
mi, _ = WfMi(mi_type='cc', inference='ffx').fit(ds, mcp=None)

# plot the result
plt.figure(figsize=(10, 8))
mi.plot(x='times', hue='roi')
plt.axvline(0., color='k')
plt.title("Information on mean activity");

ds

## 3.2 Extract the information at the single-contact level
### 3.2.1 For a single subject

In [ ]:
# load data and build the DatasetEphy
hga = load_ms(root, s_range=5, model='pe', condition='rew', space='channels')
ds = DatasetEphy(hga, y='pe', roi='channels', times='times')

# measure information
mi, _ = WfMi(mi_type='cc', inference='ffx').fit(ds, mcp=None)

# plot the result
plt.figure(figsize=(10, 8))
mi.plot(x='times', hue='roi')
plt.axvline(0., color='k')
plt.title("Information at the single contact level");

ds

### 3.2.2 For multiple subjects

In [ ]:
# load data and build the DatasetEphy
hga = load_ms(root, s_range=[3, 5], model='pe', condition='rew', space='channels')
ds = DatasetEphy(hga, y='pe', roi='channels', times='times')

# measure information
mi, _ = WfMi(mi_type='cc', inference='ffx').fit(ds, mcp=None)

# plot the result
plt.figure(figsize=(10, 8))
mi.plot(x='times', hue='roi')
plt.axvline(0., color='k')
plt.title("Information at the single contact level");

ds

## 3.3 Concatenate the activity across contacts

In [ ]:
# load data and build the DatasetEphy
hga = load_ms(root, s_range=5, model='pe', condition='rew', space='roi', mean_roi=False)
ds = DatasetEphy(hga, y='pe', roi='roi', times='times')

# measure information
mi, _ = WfMi(mi_type='cc', inference='ffx').fit(ds, mcp=None)

# plot the result
plt.figure(figsize=(10, 8))
mi.plot(x='times', hue='roi')
plt.axvline(0., color='k')
plt.title("Concatenate activity across contacts");

ds

---
# **4. Use alternative estimators of information** (Bonus)

To estimate the quantity of information shared between the brain data and an external variable, Frites uses by default metrics from the information-theory (i.e. _Gaussian Copula Mutual Information (GCMI)_). The GCMI is a good estimator because it's fast, it can detect many types of relations and it's relatively robust to the presence of noise in the data. However, more powerfull estimators exist, such as in the field of machine learning. Frites allows to provide custom estimators and also include some basic ones (like correlation).

## 4.1 Define a custom estimator

In this part, we define an estimator for computing the correlation between two continuous variables

In [ ]:
from frites.estimator import CustomEstimator

# function for computing 
def correlation(x, y):
    """Compute the correlation between two variables.
    """
    n_var, n_mv, n_samples = x.shape
    corr = np.zeros((n_var,))
    for k in range(n_var):
        corr[k] = np.corrcoef(x[k, ...], y[k, ...])[0, 1]
    return corr

# define you custom 
est = CustomEstimator('custom_correlation', 'cc', correlation,
                      multivariate=False)

## 4.2 Use this custom estimator

In [ ]:
# load data and build the DatasetEphy
hga = load_ms(root, model='pe', condition='rew', space='roi')
ds = DatasetEphy(hga, y='pe', roi='roi', times='times')

# measure information
mi, _ = WfMi(mi_type='cc', estimator=est).fit(ds, mcp=None)

# plot the result
plt.figure(figsize=(10, 8))
mi.plot(x='times', hue='roi')
plt.axvline(0., color='k')
plt.title(r"$Correlation(HGA; RPE)$");


---
# **---- Test yourself ! ----**
## **1. Data loading**
### 1.1 Load the data of a single subject

<div class="alert alert-warning"><p>

**[Instructions]**

Load the data of :
- Subject 6 only (`s_range`)
- During the reward condition (`condition`)
- For the task-related variable, use the outcome (`model`)
- The spatial dimension should be described with channel names (`space`)
</p></div>

In [ ]:
# write your answer

### 1.2 Load the data of a multiple subjects

<div class="alert alert-warning"><p>

**[Instructions]**

Same but for subjects between [6, 10]
</p></div>

In [ ]:
# write your answer

### 1.3 Switch condition and model

<div class="alert alert-warning"><p>

**[Instructions]**

Load the data of :
- Subject 6 only
- For the model and condition, use the Punishment Prediction Error
- The spatial dimension should be described with brain region names
</p></div>

In [ ]:
# write your answer

---
## **2. Measuring information**
### 2.1 Model-free analysis for a single subject

<div class="alert alert-warning"><p>

**[Instructions]**

On **subject 6**, what is the **channel** name that share the maximum of information between **outcomes** during the **reward** condition?
</p></div>

In [ ]:
# write your answer

### 2.2 Model-free analysis across all of the subjects

<div class="alert alert-warning"><p>

**[Instructions]**

Across all of the subjects, what is the name of the **brain region** that seems to better differentiate the **outcomes** during different **contexts**? (i.e. `condition='context'`)
</p></div>

In [ ]:
# write your answer

### 2.3 Model-based analysis for a single subject

<div class="alert alert-warning"><p>

**[Instructions]**

For subject 6, what is the name of the **channel** sharing the most information with the **reaction time** (`model='rt'`) during the **reward** condition?
</p></div>

In [ ]:
# write your answer

### 2.4 Model-based analysis for multiple subjects

<div class="alert alert-warning"><p>

**[Instructions]**

Across all of the subjects, what is the name of the **brain region** sharing the most information with the **punishment** (`condition`) **prediction error** (`model`)?
</p></div>

In [ ]:
# write your answer

### 2.5 Avoid taking the mean of neural activity

If you didn't change anything, the function that load multiple subjects takes the mean of the high-gamma activity per brain region (`space='roi'`). As  said before, this might not be the best way to take full benefit of the information contained at the single-contact level. In this last exercise, we are going to avoid this behavior by concatenating the activity across contacts withing a brain region.

<div class="alert alert-warning"><p>

**[Instructions]**

Across **all of the subjects**, what is the name of the **brain region** that share the maximum amount of information with the **reaction time** during the reward condition? And off course, without taking the mean of HGA within region (`mean_roi=False`) !

</p></div>

In [ ]:
# write your answer